# Amharic Hate Speech Detector using SVM

## Preprocessing

### Importing Dataset

In [1]:
import re
import math
import pandas
import string
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
%matplotlib inline

In [3]:
DATASET_FILEPATH = "datasets/dataset.xls"
raw_data = pandas.read_excel(DATASET_FILEPATH, sheet_name='sheet')

raw_data = shuffle(raw_data)
raw_data.head()

,Comment No.,0,Comment,Like,(view source),Timestamp
1520,42.0,0.0,:),0,view comment,"Monday, April 27, 2020 8:24am GMT"
269,NaN,0.0,አሁን አንተን ጊዜ አንስቶሀል ማለት ነው?፡፡ በል የአክስቶችህን ደሞዝ ን...,0,view comment,"Sunday, April 19, 2020 7:39pm GMT"
377,38.0,NaN,NaN,0,view comment,"Friday, April 3, 2020 12:19am GMT"
2514,19.0,0.0,ሰውዬው የአእምሮ ችግር አለበት ሕዝብን ሳያወያይ እንዲህ ማድረጉ የጤና አ...,0,view comment,"Thursday, April 23, 2020 5:36pm GMT"
1606,NaN,0.0,"Hahaha.....,,,,,, ere fitse",0,view comment,"Saturday, April 18, 2020 4:18pm GMT"


### Extracting Data

In [4]:
labels = raw_data.iloc[:, 1]
comments = raw_data.iloc[:, 2]

### Remove Alphabets

In [5]:
comments = [re.sub(r"[A-Za-z]", "", str(comment)) for comment in comments]

### Remove Numbers

In [6]:
comments = [re.sub('[0-9]', "", str(comment)) for comment in comments]

### Remove Punctuations

In [7]:
punctuations = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~፡።፣፥¡"
comments = [str(comment).translate(str.maketrans('', '', punctuations)) for comment in comments]

### Remove Whitespaces

In [9]:
comments = [(" ".join([ word for word in str(comment).split()])) for comment in comments]

### Remove Empty Comments

In [10]:
X = []
Y = []

number_of_comments = len(comments)
for i in range(number_of_comments):
    if comments[i].strip() and not math.isnan(float(labels[i])):
        X.append(comments[i])
        Y.append(labels[i])

## TF-IDF Encoding

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
tfidf_vectorizer = TfidfVectorizer(max_features=500, min_df=5, max_df=0.7)
X = tfidf_vectorizer.fit_transform(X).toarray()

In [13]:
print("Training Dataset Shape: ", X.shape)
print("Training Label Shape: ", len(Y))

Training Dataset Shape:  (2480, 500)
Training Label Shape:  2480


## Split Dataset

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [15]:
print("X_train Shape: ", X_train.shape)
print("X_test Shape: ", X_test.shape)

X_train Shape:  (1984, 500)
X_test Shape:  (496, 500)


In [16]:
print("Y_train Shape: ", len(Y_train))
print("Y_test Shape: ", len(Y_test))

Y_train Shape:  1984
Y_test Shape:  496


## SVM

In [17]:
from sklearn.svm import SVC

In [18]:
classifier = SVC(kernel='linear')
classifier.fit(X_train, Y_train) 

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## Evaluate Model

In [19]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [20]:
Y_pred = classifier.predict(X_test)

### Confusion Matrix

In [21]:
print(confusion_matrix(Y_test, Y_pred))

[[483   0]
 [ 13   0]]


### Classification Report

In [22]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99       483
         1.0       0.00      0.00      0.00        13

    accuracy                           0.97       496
   macro avg       0.49      0.50      0.49       496
weighted avg       0.95      0.97      0.96       496



/home/biruk/miniconda3/envs/amharic-hate-speech-detector/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Accuracy Score

In [23]:
print(accuracy_score(Y_test, Y_pred))

0.9737903225806451
